In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import weave

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def lr_schedule(epoch):
    return (0.00100385 - 6.92308e-6*epoch)

def straight_stack(x, num_filters, rep = 3, weight_decay = 1e-4, dropout_start = 0.2, dropout_add = .1):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='elu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='elu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(dropout_start+n*dropout_add)(x)
    return x

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

batch_size = 64

(50000, 3, 32, 32)


In [4]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
                                     histogram_freq=0, 
                                     batch_size=batch_size, 
                                     write_graph=True)
num_filters = 16
num_val_test = 500
epochs = 1
model_details = {}
for num_stack in [1, 2, 4, 8, 16]:
    inputs = Input(shape=(3,32,32))
    input_array = []
    for stack in range(num_stack):
        x = straight_stack(inputs, num_filters/num_stack)
        input_array.append(x)
    pyrm_layers = np.log2(num_stack)
    if pyrm_layers > 0:
        x = weave.pyrm_net(input_size=(4,4,128),
                           n_layers = pyrm_layers,
                           n_filters_start = 128,
                           n_gpus = 1,
                           inputs = input_array,
                           r_filter = 1,
                           r_combine= 1,
                           min_dim = 2,
                           max_pool_loc = 5,
                           pure_combine = True,
                           end_max_pool = False)
    else:
        pass
    
    x = Flatten()(x)
    
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    num_params = model.count_params()
    
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=10,#x_train.shape[0] // batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule), viewer])
    model_details[num_stack] = (num_params, model.evaluate(x_test[0:10],y_test[0:10]))
    
    
    

Epoch 1/1
10/10 [==============================] - 0s 11ms/step
GPU settings allow for inf layers
Minimum output size allow for 5 layers
Number of layers 1
First Layer Size: 1 (Number of Units)
('Disjoint:', True)
number of units 1
number of devices 2
number of inputs 2
First Layer Output Size: 1 (Number of Tensors)
Final Layer Size 1
Epoch 1/1
10/10 [==============================] - 0s 12ms/step
GPU settings allow for inf layers
Minimum output size allow for 5 layers
Number of layers 2
First Layer Size: 2 (Number of Units)
('Disjoint:', True)
number of units 2
number of devices 4
number of inputs 4
First Layer Output Size: 2 (Number of Tensors)
('Disjoint:', True)
number of units 1
number of devices 4
number of inputs 2
Final Layer Size 1
Epoch 1/1
10/10 [==============================] - 0s 14ms/step
GPU settings allow for inf layers
Minimum output size allow for 5 layers
Number of layers 3
First Layer Size: 4 (Number of Units)
('Disjoint:', True)
number of units 4
number of devices

In [5]:
model_details


{1: (82778, [2.536757469177246, 0.10000000149011612]),
 2: (94746, [1.9628959894180298, 0.5]),
 4: (225498, [1.9888815879821777, 0.4000000059604645]),
 8: (513786, [2.167442798614502, 0.20000000298023224]),
 16: (1103754, [2.275731086730957, 0.0])}